# Scale number of GPUs while keeping the CPU number the same

In [3]:
import json
import glob
import numpy as np

In [4]:
benchmark_paths = {
    'candle': '/work/li.baol/GIT/candle_benchmarks/benchmark_logs',
    'cv': '/work/li.baol/GIT/examples/imagenet/benchmark_logs',
    'nlp': '/work/li.baol/GIT/transformers/examples/pytorch/question-answering/benchmark_logs'
}

upgrade_options = [1, 2, 4]

perf_scale = {} # {'cv': {1x: x, 2x: x, 4x: x}}
carbon_scale = {}

for benchmark in benchmark_paths:
    perf_scale[benchmark] = {}
    carbon_scale[benchmark] = {}
    for scale in upgrade_options:
        read_path_carbon = f'{benchmark_paths[benchmark]}/{scale}xv100/carbon_*.json'
        read_list_carbon = glob.glob(read_path_carbon)
        read_path_perf = f'{benchmark_paths[benchmark]}/{scale}xv100/time_*.json'
        read_list_perf = glob.glob(read_path_perf)        
        perf_scale[benchmark][scale] = []
        carbon_scale[benchmark][scale] = []
        for read in read_list_perf:
            with open(read) as f:
                load = json.load(f)
            perf_scale[benchmark][scale].append(np.mean(load[1:])/scale)
        for read in read_list_carbon:
            with open(read) as f:
                load = json.load(f)
            carbon_scale[benchmark][scale].append(np.mean(load['energy (kWh)'])/scale)
    norm_perf = np.array(perf_scale[benchmark][1])
    norm_carbon = np.array(carbon_scale[benchmark][1])    
    for scale in upgrade_options:
        perf_scale[benchmark][scale] = np.mean(np.array(perf_scale[benchmark][scale]) / norm_perf)
        carbon_scale[benchmark][scale] = np.mean(np.array(carbon_scale[benchmark][scale]) / norm_carbon)
with open('iteration_time_vs_num_gpu.json', 'w') as f:
    json.dump(perf_scale, f, indent=4)        
with open('operational_carbon_vs_num_gpu.json', 'w') as f:
    json.dump(carbon_scale, f, indent=4)            